In [3]:
# Import necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
# Generate 40 small support cases with labels
data = [
    {'text': 'The motor is overheating after prolonged use.', 'label': 'motor'},
    {'text': 'Printhead alignment is off, causing blurry prints.', 'label': 'printhead'},
    {'text': 'The printer shows an unknown error code.', 'label': 'other'},
    {'text': 'Motor makes a grinding noise during operation.', 'label': 'motor'},
    {'text': 'Printhead nozzles are clogged and need cleaning.', 'label': 'printhead'},
    {'text': 'Cannot connect the printer to Wi-Fi.', 'label': 'other'},
    {'text': 'Motor fails to start when initiating a print job.', 'label': 'motor'},
    {'text': 'Printhead alignment utility fails every time.', 'label': 'printhead'},
    {'text': 'Paper jams occur frequently in the tray.', 'label': 'other'},
    {'text': 'Motor belt is worn out and slips.', 'label': 'motor'},
    {'text': 'Printhead produces streaks across all prints.', 'label': 'printhead'},
    {'text': 'The touch screen is unresponsive.', 'label': 'other'},
    {'text': 'Motor overheats and shuts down.', 'label': 'motor'},
    {'text': 'Printhead is not recognized by the printer.', 'label': 'printhead'},
    {'text': 'Printer driver installation fails.', 'label': 'other'},
    {'text': 'Motor emits a burning smell during use.', 'label': 'motor'},
    {'text': 'Printhead colors are inaccurate and faded.', 'label': 'printhead'},
    {'text': 'Cannot update printer firmware.', 'label': 'other'},
    {'text': 'Motor vibrates excessively when printing.', 'label': 'motor'},
    {'text': 'Printhead cleaning cycle does not complete.', 'label': 'printhead'},
    {'text': 'The motor is not responding to commands.', 'label': 'motor'},
    {'text': 'Printhead is scratching the paper surface.', 'label': 'printhead'},
    {'text': 'Unable to print from mobile devices.', 'label': 'other'},
    {'text': 'Motor speed is inconsistent during operation.', 'label': 'motor'},
    {'text': 'Printhead calibration fails to improve quality.', 'label': 'printhead'},
    {'text': 'Error message says "cartridge not recognized".', 'label': 'other'},
    {'text': 'Motor bearings are worn out.', 'label': 'motor'},
    {'text': 'Printhead drips ink even when idle.', 'label': 'printhead'},
    {'text': 'Printer disconnects from network intermittently.', 'label': 'other'},
    {'text': 'Motor consumes excessive power.', 'label': 'motor'},
    {'text': 'Printhead alignment is perfect but colors are off.', 'label': 'printhead'},
    {'text': 'Cannot access printer settings via control panel.', 'label': 'other'},
    {'text': 'Motor controller board needs replacement.', 'label': 'motor'},
    {'text': 'Printhead cleaning does not resolve smudging.', 'label': 'printhead'},
    {'text': 'Printer tray does not hold paper properly.', 'label': 'other'},
    {'text': 'Motor makes unusual humming sounds.', 'label': 'motor'},
    {'text': 'Printhead requires frequent replacement.', 'label': 'printhead'},
    {'text': 'Cannot perform duplex printing.', 'label': 'other'},
    {'text': 'Motor wiring harness is damaged.', 'label': 'motor'},
    {'text': 'Printhead temperature exceeds normal levels.', 'label': 'printhead'},
]

# Map labels to integers
label_mapping = {'motor': 0, 'printhead': 1, 'other': 2}
texts = [item['text'] for item in data]
labels = [label_mapping[item['label']] for item in data]

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Generate 10 test examples for prediction
test_data = [
    'Printhead leaks ink onto the paper.',
    'Motor starts but then stops immediately.',
    'Unable to scan documents to email.',
    'Motor shaft is misaligned and needs adjustment.',
    'Printhead alignment successful but quality is poor.',
    'Cannot connect to the printer via Bluetooth.',
    'Motor overheating error appears after startup.',
    'Printhead produces faded text on the left side.',
    'Paper feeder is not pulling paper correctly.',
    'Motor drive belt snapped during operation.',
]

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Tokenize the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Create a custom Dataset class
class SupportCasesDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


# Prepare the datasets
train_dataset = SupportCasesDataset(train_encodings, train_labels)
val_dataset = SupportCasesDataset(val_encodings, val_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=10,             # Number of training epochs
    per_device_train_batch_size=4,   # Batch size per device
    evaluation_strategy='epoch',     # Evaluate at each epoch
    logging_dir='./logs',            # Logging directory
    logging_steps=10,
    seed=42,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")

# Prepare the test data
test_encodings = tokenizer(test_data, truncation=True, padding=True)
test_dataset = SupportCasesDataset(test_encodings, [0]*len(test_data))  # Dummy labels

# Make predictions
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Map predicted labels back to class names
inv_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_classes = [inv_label_mapping[label] for label in pred_labels]

# Print the predictions
for text, pred_class in zip(test_data, predicted_classes):
    print(f'Text: "{text}"')
    print(f'Predicted class: {pred_class}')
    print('---')



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\enric\Documents\projects\play_with_transformers\.venv\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                              
 10%|█         | 8/80 [00:19<02:37,  2.19s/it]

{'eval_loss': 0.8828052878379822, 'eval_accuracy': 0.625, 'eval_precision': 0.85, 'eval_recall': 0.625, 'eval_f1': 0.6303571428571428, 'eval_runtime': 0.2653, 'eval_samples_per_second': 30.15, 'eval_steps_per_second': 3.769, 'epoch': 1.0}


 12%|█▎        | 10/80 [00:23<02:25,  2.07s/it]

{'loss': 0.9949, 'grad_norm': 8.293554306030273, 'learning_rate': 4.375e-05, 'epoch': 1.25}


 20%|██        | 16/80 [00:35<01:59,  1.86s/it]

...

{'eval_loss': 0.00618194742128253, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.2497, 'eval_samples_per_second': 32.042, 'eval_steps_per_second': 4.005, 'epoch': 10.0}
{'train_runtime': 179.4024, 'train_samples_per_second': 1.784, 'train_steps_per_second': 0.446, 'train_loss': 0.23349415017291902, 'epoch': 10.0}


100%|██████████| 1/1 [00:00<00:00, 333.36it/s]


Validation Loss: 0.0062


100%|██████████| 2/2 [00:00<00:00, 13.49it/s]c:\Users\enric\Documents\projects\play_with_transformers\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 2/2 [00:00<00:00, 12.70it/s]

Text: "Printhead leaks ink onto the paper."
Predicted class: printhead
---
Text: "Motor starts but then stops immediately."
Predicted class: motor
---
Text: "Unable to scan documents to email."
Predicted class: other
---
Text: "Motor shaft is misaligned and needs adjustment."
Predicted class: motor
---
Text: "Printhead alignment successful but quality is poor."
Predicted class: printhead
---
Text: "Cannot connect to the printer via Bluetooth."
Predicted class: other
---
Text: "Motor overheating error appears after startup."
Predicted class: motor
---
Text: "Printhead produces faded text on the left side."
Predicted class: printhead
---
Text: "Paper feeder is not pulling paper correctly."
Predicted class: other
---
Text: "Motor drive belt snapped during operation."
Predicted class: motor
---


In [5]:
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall: {eval_results['eval_recall']:.4f}")
print(f"F1 Score: {eval_results['eval_f1']:.4f}")

100%|██████████| 1/1 [00:00<00:00, 249.50it/s]

Validation Loss: 0.0062
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
